In [1]:
from utils.graph import WeightedGraphAdjList as g

In [2]:
gr = g()

In [3]:
gr.insert_weighted_node(0, 1, 4)
gr.insert_weighted_node(0, 7, 8)
gr.insert_weighted_node(1, 2, 8)
gr.insert_weighted_node(1, 7, 11)
gr.insert_weighted_node(2, 3, 7)
gr.insert_weighted_node(2, 8, 2)
gr.insert_weighted_node(2, 5, 4)
gr.insert_weighted_node(3, 4, 9)
gr.insert_weighted_node(3, 5, 14)
gr.insert_weighted_node(4, 5, 10)
gr.insert_weighted_node(5, 6, 2)
gr.insert_weighted_node(6, 7, 1)
gr.insert_weighted_node(6, 8, 6)
gr.insert_weighted_node(7, 8, 7)

In [4]:
gr.graph

[[0, 1, 4],
 [0, 7, 8],
 [1, 2, 8],
 [1, 7, 11],
 [2, 3, 7],
 [2, 8, 2],
 [2, 5, 4],
 [3, 4, 9],
 [3, 5, 14],
 [4, 5, 10],
 [5, 6, 2],
 [6, 7, 1],
 [6, 8, 6],
 [7, 8, 7]]

In [5]:
class MinHeapPQ:
    def __init__(self):
        self.heap = []
        self.size = 0
        self.position = []

    def insert(self, key, weight):
        self.heap.append([key, weight])
        self.position.append(key)
        n = (self.size // 2) - 1
        for node in range(n, -1, -1):
            self.min_heapify(node)

    def swap(self, idx_a, idx_b):
        self.heap[idx_a], self.heap[idx_b] = (self.heap[idx_b], self.heap[idx_a])

    def min_heapify(self, parent):
        smallest = parent
        left_child = 2 * parent + 1
        right_child = 2 * parent + 2

        if left_child < self.heap_size and self.heap[parent] > self.heap[left_child]:
            smallest = left_child
        if (
            right_child < self.heap_size
            and self.heap[smallest] > self.heap[right_child]
        ):
            smallest = right_child

        if smallest != parent:
            self.swap(smallest, parent)
            # heapify again the one which has been moved
            self.min_heapify(smallest)

In [6]:
# A Python program for Prims's MST for
# adjacency list representation of graph

from collections import defaultdict
import sys


class Heap:
    def __init__(self):
        self.array = []
        self.size = 0
        self.pos = []

    def newMinHeapNode(self, v, dist):
        minHeapNode = [v, dist]
        return minHeapNode

    # A utility function to swap two nodes of
    # min heap. Needed for min heapify
    def swapMinHeapNode(self, a, b):
        t = self.array[a]
        self.array[a] = self.array[b]
        self.array[b] = t

    # A standard function to heapify at given idx
    # This function also updates position of nodes
    # when they are swapped. Position is needed
    # for decreaseKey()
    def minHeapify(self, idx):
        smallest = idx
        left = 2 * idx + 1
        right = 2 * idx + 2

        if left < self.size and self.array[left][1] < self.array[smallest][1]:
            smallest = left

        if right < self.size and self.array[right][1] < self.array[smallest][1]:
            smallest = right

        # The nodes to be swapped in min heap
        # if idx is not smallest
        if smallest != idx:

            # Swap positions
            self.pos[self.array[smallest][0]] = idx
            self.pos[self.array[idx][0]] = smallest

            # Swap nodes
            self.swapMinHeapNode(smallest, idx)

            self.minHeapify(smallest)

    # Standard function to extract minimum node from heap
    def extractMin(self):

        # Return NULL wif heap is empty
        if self.isEmpty() == True:
            return

        # Store the root node
        root = self.array[0]

        # Replace root node with last node
        lastNode = self.array[self.size - 1]
        self.array[0] = lastNode

        # Update position of last node
        self.pos[lastNode[0]] = 0
        self.pos[root[0]] = self.size - 1

        # Reduce heap size and heapify root
        self.size -= 1
        self.minHeapify(0)

        return root

    def isEmpty(self):
        return True if self.size == 0 else False

    def decreaseKey(self, v, dist):

        # Get the index of v in heap array

        i = self.pos[v]

        # Get the node and update its dist value
        self.array[i][1] = dist

        # Travel up while the complete tree is not
        # heapified. This is a O(Logn) loop
        while i > 0 and self.array[i][1] < self.array[(i - 1) // 2][1]:

            print("array: ", self.array)
            print("pos: ", self.pos)
            # Swap this node with its parent
            self.pos[self.array[i][0]] = (i - 1) / 2
            self.pos[self.array[(i - 1) // 2][0]] = i
            self.swapMinHeapNode(i, (i - 1) // 2)

            # move to parent index
            i = (i - 1) // 2

    # A utility function to check if a given vertex
    # 'v' is in min heap or not
    def isInMinHeap(self, v):

        if self.pos[v] < self.size:
            return True
        return False


def printArr(parent, n):
    for i in range(1, n):
        print("% d - % d" % (parent[i], i))


class Graph:
    def __init__(self, V):
        self.V = V
        self.graph = defaultdict(list)

    # Adds an edge to an undirected graph
    def addEdge(self, src, dest, weight):

        # Add an edge from src to dest. A new node is
        # added to the adjacency list of src. The node
        # is added at the beginning. The first element of
        # the node has the destination and the second
        # elements has the weight
        newNode = [dest, weight]
        self.graph[src].insert(0, newNode)

        # Since graph is undirected, add an edge from
        # dest to src also
        newNode = [src, weight]
        self.graph[dest].insert(0, newNode)

    # The main function that prints the Minimum
    # Spanning Tree(MST) using the Prim's Algorithm.
    # It is a O(ELogV) function
    def PrimMST(self):
        # Get the number of vertices in graph
        V = self.V

        # key values used to pick minimum weight edge in cut
        key = []

        # List to store constructed MST
        parent = []

        # minHeap represents set E
        minHeap = Heap()

        # Initialize min heap with all vertices. Key values of all
        # vertices (except the 0th vertex) is is initially infinite
        for v in range(V):
            parent.append(-1)
            key.append(1e2)
            minHeap.array.append(minHeap.newMinHeapNode(v, key[v]))
            minHeap.pos.append(v)

        print("minheap.array: ", minHeap.array)
        print("minHeap.pos: ", minHeap.pos)
        print()

        # Make key value of 0th vertex as 0 so
        # that it is extracted first
        minHeap.pos[0] = 0
        key[0] = 0
        minHeap.decreaseKey(0, key[0])

        # Initially size of min heap is equal to V
        minHeap.size = V

        print("minheap.array: ", minHeap.array)
        print("minHeap.pos: ", minHeap.pos)
        print()

        # In the following loop, min heap contains all nodes
        # not yet added in the MST.
        while minHeap.isEmpty() == False:

            # Extract the vertex with minimum distance value
            newHeapNode = minHeap.extractMin()
            u = newHeapNode[0]

            # Traverse through all adjacent vertices of u
            # (the extracted vertex) and update their
            # distance values
            for pCrawl in self.graph[u]:

                v = pCrawl[0]

                # If shortest distance to v is not finalized
                # yet, and distance to v through u is less than
                # its previously calculated distance
                if minHeap.isInMinHeap(v) and pCrawl[1] < key[v]:
                    key[v] = pCrawl[1]
                    parent[v] = u

                    # update distance value in min heap also
                    minHeap.decreaseKey(v, key[v])

        printArr(parent, V)


# Driver program to test the above functions
graph = Graph(5)
graph.addEdge(0, 1, 2)
graph.addEdge(0, 3, 6)
graph.addEdge(1, 2, 3)
graph.addEdge(1, 3, 8)
graph.addEdge(1, 4, 5)
graph.addEdge(2, 4, 7)
graph.addEdge(3, 4, 9)


# graph.addEdge(0, 1, 4)
# graph.addEdge(0, 7, 8)
# graph.addEdge(1, 2, 8)
# graph.addEdge(1, 7, 11)
# graph.addEdge(2, 3, 7)
# graph.addEdge(2, 8, 2)
# graph.addEdge(2, 5, 4)
# graph.addEdge(3, 4, 9)
# graph.addEdge(3, 5, 14)
# graph.addEdge(4, 5, 10)
# graph.addEdge(5, 6, 2)
# graph.addEdge(6, 7, 1)
# graph.addEdge(6, 8, 6)
# graph.addEdge(7, 8, 7)
graph.PrimMST()

# This code is contributed by Divyanshu Mehta

minheap.array:  [[0, 100.0], [1, 100.0], [2, 100.0], [3, 100.0], [4, 100.0]]
minHeap.pos:  [0, 1, 2, 3, 4]

minheap.array:  [[0, 0], [1, 100.0], [2, 100.0], [3, 100.0], [4, 100.0]]
minHeap.pos:  [0, 1, 2, 3, 4]

array:  [[4, 100.0], [1, 100.0], [2, 100.0], [3, 6], [4, 100.0]]
pos:  [4, 1, 2, 3, 0]
array:  [[4, 100.0], [3, 6], [2, 100.0], [1, 100.0], [4, 100.0]]
pos:  [4, 3, 2, 1.0, 0]
array:  [[3, 6], [4, 100.0], [2, 100.0], [1, 2], [4, 100.0]]
pos:  [4, 3, 2, 0.0, 1]
array:  [[3, 6], [1, 2], [2, 100.0], [4, 100.0], [4, 100.0]]
pos:  [4, 1.0, 2, 0.0, 3]
array:  [[3, 6], [4, 5], [2, 100.0], [4, 5], [4, 5]]
pos:  [4, 3, 2, 0, 1]
array:  [[4, 5], [3, 6], [2, 3], [4, 5], [4, 5]]
pos:  [4, 3, 2, 1, 0.0]
 0 -  1
 1 -  2
 0 -  3
 1 -  4


 0 -  1

 5 -  2
 
 2 -  3
 
 3 -  4
 
 
 6 -  5
 
 7 -  6
 
 0 -  7
 
 2 -  8

In [7]:
graph.graph

defaultdict(list,
            {0: [[3, 6], [1, 2]],
             1: [[4, 5], [3, 8], [2, 3], [0, 2]],
             3: [[4, 9], [1, 8], [0, 6]],
             2: [[4, 7], [1, 3]],
             4: [[3, 9], [2, 7], [1, 5]]})

In [46]:
class MinHeapPriorityQueue:
    def __init__(self, arr=None):
        self.heap = []
        self.heap_size = 0
        self.pos = []
        if arr is not None:
            self.heap = arr
            self.heap_size = len(self.heap)
            self.pos = list(range(len(arr)))

    def min_heapify(self, parent):
        smallest = parent
        left_child = 2 * parent + 1
        right_child = 2 * parent + 2

        if (
            left_child < self.heap_size
            and self.heap[parent][1] > self.heap[left_child][1]
        ):
            smallest = left_child
        if (
            right_child < self.heap_size
            and self.heap[smallest][1] > self.heap[right_child][1]
        ):
            smallest = right_child

        if smallest != parent:
            self.pos[smallest], self.pos[parent] = self.pos[parent], self.pos[smallest]

            self.swap(smallest, parent)
            # heapify again the one which has been moved
            self.min_heapify(smallest)

    def insert_min_heap(self, key, weight):
        self.heap.append([key, weight])
        self.pos.append(self.heap_size)
        self.heap_size += 1

        n = (self.heap_size // 2) - 1
        for node in range(n, -1, -1):
            self.min_heapify(node)

    def swap(self, idx_a, idx_b):
        self.heap[idx_a], self.heap[idx_b] = (self.heap[idx_b], self.heap[idx_a])

    def build_min_heap(self):
        n = (self.heap_size // 2) - 1
        for node in range(n, -1, -1):
            self.min_heapify(node)

    # def print_root(self):
    #     print(self.heap[0])

    # def get_max_root_method_1(self):
    #     if self.heap_size > 0:
    #         element = self.heap.pop(0)
    #         self.heap_size -= 1
    #         self.max_heapify(0)
    #         return element
    #     else:
    #         print("Heap is empty")

    # def get_max_root_method_2(self):
    #     if self.heap_size > 0:
    #         element = self.heap[0]
    #         self.heap[0] = self.heap[self.heap_size - 1]
    #         del self.heap[self.heap_size - 1]
    #         self.heap_size -= 1
    #         self.max_heapify(0)
    #         return element
    #     else:
    #         print("Heap is empty")

    def get_min_root(self):
        if self.heap_size > 0:

            element = self.heap.pop(0)
            idx = element[0]
            self.pos.remove(idx)
            self.heap_size -= 1
            self.min_heapify(0)
            return element
        else:
            print("Heap is empty")

    # # * In min heap pq, increase key means push down
    # # * push down means heapify
    # # * heapify means min_heapify
    # def increase_key(self, index, key):
    #     self.heap[index] = key
    #     self.min_heapify(index)

    # In min heap pq, decrease key means push up
    # push up means swap current node with parent
    # do ... until the node is greater than the parent
    # when we are pushing up, we will always move towards zero index
    # or root
    def decrease_key(self, index, key):
        # get position of index in pos array
        poi = self.pos.index(index)

        self.heap[poi][1] = key

        while (poi > 0) and self.heap[(poi - 1) // 2][1] > self.heap[poi][1]:
            self.pos[poi], self.pos[(poi - 1) // 2] = (
                self.pos[(poi - 1) // 2],
                self.pos[poi],
            )
            self.swap(poi, (poi - 1) // 2)
            poi = (poi - 1) // 2

    def is_empty(self):
        return True if self.heap_size == 0 else False

    # def is_in_heap(self, v):
    #     if v in self.pos:
    #         if self.pos.index(v) < self.heap_size:
    #             return True
    #     return False

    def is_in_heap(self, v):
        if self.pos.index(v) < self.heap_size:
            return True
        return False

In [47]:
heap = MinHeapPriorityQueue()

In [48]:
heap.insert_min_heap(0, 10)  # 0
heap.insert_min_heap(1, 7)  # 1
heap.insert_min_heap(2, 4)  # 2
heap.insert_min_heap(3, 5)  # 3
heap.insert_min_heap(4, 3)  # 4

In [49]:
heap.heap

[[4, 3], [2, 4], [1, 7], [0, 10], [3, 5]]

In [50]:
heap.is_empty()

False

In [51]:
# for i in range(5):
#     node = heap.get_min_root()
#     print(node)

In [52]:
heap.pos

[4, 2, 1, 0, 3]

In [53]:
heap.pos.index(0)

3

In [54]:
heap.is_in_heap(5)

False

In [55]:
heap.decrease_key(0, 1)

In [56]:
heap.heap

[[0, 1], [4, 3], [1, 7], [2, 4], [3, 5]]

In [57]:
heap.pos

[0, 4, 1, 2, 3]

In [58]:
heap.decrease_key(3, 2)

In [59]:
heap.heap

[[0, 1], [3, 2], [1, 7], [2, 4], [4, 3]]

In [60]:
heap.pos

[0, 3, 1, 2, 4]

In [61]:
for i in range(5):
    node = heap.get_min_root()
    print("node: ", node)
    # print(heap.pos[node[0]])
    # print("pos: ", heap.pos)

node:  [0, 1]
node:  [3, 2]
node:  [4, 3]
node:  [2, 4]
node:  [1, 7]


In [24]:
heap.heap_size

0

In [25]:
heap.is_empty()

True

In [26]:
def computeMST(graph):
    key = {}
    pred = {}
    inqueue = {}

    for v in graph:
        key[v] = 100000
        pred[v] = None

    key[v] = 0
    pq = MinHeapPriorityQueue()

    for v in graph:
        pq.insert_min_heap(v, key[v])
        # pq.heap.append([v, key[v]])
        inqueue[v] = True

    while not pq.is_empty():
        u = pq.get_min_root()[0]
        # print(u)
        # print(inqueue)
        inqueue[u] = False

        for v in graph[u]:
            if inqueue[v]:
                wt = graph[u][v]
                if wt < key[v]:
                    pred[v] = u
                    key[v] = wt
                    pq.decrease_key(v, wt)
    return pred

In [27]:
# from collections import defaultdict
# class Graph():

#     # def __init__(self, V):
#     #     self.V = V
#     #     self.graph = defaultdict(dict)

#     # Adds an edge to an undirected graph
#     # def addEdge(self, src, dest, weight):

#     #     # Add an edge from src to dest.  A new node is
#     #     # added to the adjacency list of src. The node
#     #     # is added at the beginning. The first element of
#     #     # the node has the destination and the second
#     #     # elements has the weight
#     #     newNode = [dest, weight]
#     #     self.graph[src].insert(0, newNode)

#     #     # Since graph is undirected, add an edge from
#     #     # dest to src also
#     #     newNode = [src, weight]
#     #     self.graph[dest].insert(0, newNode)

#     def __init__(self):
#         self.graph = defaultdict(dict)

#     def insert_weighted_node(self, u, v, w):
#         self.graph[u][v] = w
#         self.graph[v][u] = w

In [28]:
# graph = Graph()
# graph.insert_weighted_node(0, 1, 2)
# graph.insert_weighted_node(0, 3, 8)
# graph.insert_weighted_node(0, 4, 4)

# graph.insert_weighted_node(1, 0, 2)
# graph.insert_weighted_node(1, 2, 3)

# graph.insert_weighted_node(2, 1, 3)
# graph.insert_weighted_node(2, 3, 5)
# graph.insert_weighted_node(2, 4, 1)

# graph.insert_weighted_node(3, 0, 8)
# graph.insert_weighted_node(3, 2, 5)
# graph.insert_weighted_node(3, 4, 7)

# graph.insert_weighted_node(4, 0, 4)
# graph.insert_weighted_node(4, 2, 1)
# graph.insert_weighted_node(4, 3, 7)


# # graph = Graph()
# # graph.insert_weighted_node(0, 1, 2)
# # graph.insert_weighted_node(0, 3, 6)
# # graph.insert_weighted_node(1, 2, 3)
# # graph.insert_weighted_node(1, 3, 8)
# # graph.insert_weighted_node(1, 4, 5)
# # graph.insert_weighted_node(2, 4, 7)
# # graph.insert_weighted_node(3, 4, 9)
# #graph.insert_weighted_node(4, )
# #graph.insert_weighted_node()
# #graph.insert_weighted_node()


# # graph = Graph()
# # graph.insert_weighted_node(0, 1, 4)
# # graph.insert_weighted_node(0, 7, 8)
# # graph.insert_weighted_node(1, 2, 8)
# # graph.insert_weighted_node(1, 7, 11)
# # graph.insert_weighted_node(2, 3, 7)
# # graph.insert_weighted_node(2, 8, 2)
# # graph.insert_weighted_node(2, 5, 4)
# # graph.insert_weighted_node(3, 4, 9)
# # graph.insert_weighted_node(3, 5, 14)
# # graph.insert_weighted_node(4, 5, 10)
# # graph.insert_weighted_node(5, 6, 2)
# # graph.insert_weighted_node(6, 7, 1)
# # graph.insert_weighted_node(6, 8, 6)
# # graph.insert_weighted_node(7, 8, 7)

In [29]:
from collections import defaultdict


class Graph:
    def __init__(self, V):
        self.V = V
        self.graph = defaultdict(list)

    # Adds an edge to an undirected graph
    def addEdge(self, src, dest, weight):

        # Add an edge from src to dest.  A new node is
        # added to the adjacency list of src. The node
        # is added at the beginning. The first element of
        # the node has the destination and the second
        # elements has the weight
        newNode = [dest, weight]
        self.graph[src].insert(0, newNode)

        # Since graph is undirected, add an edge from
        # dest to src also
        newNode = [src, weight]
        self.graph[dest].insert(0, newNode)

In [30]:
graph = Graph(9)
graph.addEdge(0, 1, 4)
graph.addEdge(0, 7, 8)
graph.addEdge(1, 2, 8)
graph.addEdge(1, 7, 11)
graph.addEdge(2, 3, 7)
graph.addEdge(2, 8, 2)
graph.addEdge(2, 5, 4)
graph.addEdge(3, 4, 9)
graph.addEdge(3, 5, 14)
graph.addEdge(4, 5, 10)
graph.addEdge(5, 6, 2)
graph.addEdge(6, 7, 1)
graph.addEdge(6, 8, 6)
graph.addEdge(7, 8, 7)

In [31]:
graph.graph

defaultdict(list,
            {0: [[7, 8], [1, 4]],
             1: [[7, 11], [2, 8], [0, 4]],
             7: [[8, 7], [6, 1], [1, 11], [0, 8]],
             2: [[5, 4], [8, 2], [3, 7], [1, 8]],
             3: [[5, 14], [4, 9], [2, 7]],
             8: [[7, 7], [6, 6], [2, 2]],
             5: [[6, 2], [4, 10], [3, 14], [2, 4]],
             4: [[5, 10], [3, 9]],
             6: [[8, 6], [7, 1], [5, 2]]})

In [32]:
# ans = computeMST(graph.graph)

In [33]:
# def sol(pred):
#     edges = []
#     for v in pred:
#         if pred[v] is not None:
#             edges.append((pred[v], v))
#     return edges

In [34]:
# sol(ans)

In [35]:
# ans

In [36]:
def print_arr(parent, n):
    for i in range(1, n):
        print("%s = %s" % (parent[i], i))

In [37]:
def test(graph, v):

    key = []
    parent = []

    min_heap = MinHeapPriorityQueue()

    for vx in range(v):
        parent.append(-1)
        key.append(1e7)
        min_heap.heap.append([vx, key[vx]])
        min_heap.pos.append(vx)

    min_heap.pos[0] = 0
    key[0] = 0
    min_heap.decrease_key(0, key[0])

    min_heap.heap_size = v

    # print(min_heap.heap)
    # print(min_heap.pos)

    while min_heap.is_empty() == False:
        # Extract the vertex with minimum distance value
        newHeapNode = min_heap.get_min_root()
        u = newHeapNode[0]

        for pCrawl in graph[u]:

            print(pCrawl)
            vu = pCrawl[0]

            # If shortest distance to v is not finalized
            # yet, and distance to v through u is less than
            # its previously calculated distance
            if min_heap.is_in_heap(vu) and pCrawl[1] < key[vu]:
                key[vu] = pCrawl[1]
                parent[vu] = u

                # update distance value in min heap also
                min_heap.decrease_key(vu, key[vu])

    print_arr(parent, v)

In [38]:
test(graph.graph, v=5)

[7, 8]
[1, 4]
[7, 11]
[2, 8]
[0, 4]
[5, 4]
[8, 2]
[3, 7]
[1, 8]
[5, 14]
[4, 9]
[2, 7]
[5, 10]
[3, 9]
0 = 1
1 = 2
2 = 3
3 = 4


In [39]:
for p in graph.graph[2]:
    print(p)

[5, 4]
[8, 2]
[3, 7]
[1, 8]


In [40]:
# A Python program for Prims's MST for
# adjacency list representation of graph

from collections import defaultdict
import sys


class Heap:
    def __init__(self):
        self.array = []
        self.size = 0
        self.pos = []

    def newMinHeapNode(self, v, dist):
        minHeapNode = [v, dist]
        return minHeapNode

    # A utility function to swap two nodes of
    # min heap. Needed for min heapify
    def swapMinHeapNode(self, a, b):
        t = self.array[a]
        self.array[a] = self.array[b]
        self.array[b] = t

    # A standard function to heapify at given idx
    # This function also updates position of nodes
    # when they are swapped. Position is needed
    # for decreaseKey()
    def minHeapify(self, idx):
        smallest = idx
        left = 2 * idx + 1
        right = 2 * idx + 2

        if left < self.size and self.array[left][1] < self.array[smallest][1]:
            smallest = left

        if right < self.size and self.array[right][1] < self.array[smallest][1]:
            smallest = right

        # The nodes to be swapped in min heap
        # if idx is not smallest
        if smallest != idx:

            # Swap positions
            self.pos[self.array[smallest][0]] = idx
            self.pos[self.array[idx][0]] = smallest

            # Swap nodes
            self.swapMinHeapNode(smallest, idx)

            self.minHeapify(smallest)

    # Standard function to extract minimum node from heap
    def extractMin(self):

        # Return NULL wif heap is empty
        if self.isEmpty() == True:
            return

        # Store the root node
        root = self.array[0]

        # Replace root node with last node
        lastNode = self.array[self.size - 1]
        self.array[0] = lastNode

        # Update position of last node
        self.pos[lastNode[0]] = 0
        self.pos[root[0]] = self.size - 1

        # Reduce heap size and heapify root
        self.size -= 1
        self.minHeapify(0)

        return root

    def isEmpty(self):
        return True if self.size == 0 else False

    def decreaseKey(self, v, dist):

        # Get the index of v in heap array

        i = self.pos[v]

        # Get the node and update its dist value
        self.array[i][1] = dist

        # Travel up while the complete tree is not
        # heapified. This is a O(Logn) loop
        while i > 0 and self.array[i][1] < self.array[(i - 1) // 2][1]:

            # Swap this node with its parent
            self.pos[self.array[i][0]] = (i - 1) / 2
            self.pos[self.array[(i - 1) // 2][0]] = i
            self.swapMinHeapNode(i, (i - 1) // 2)

            # move to parent index
            i = (i - 1) // 2

    # A utility function to check if a given vertex
    # 'v' is in min heap or not
    def isInMinHeap(self, v):

        if self.pos[v] < self.size:
            return True
        return False


def printArr(parent, n):
    for i in range(1, n):
        print("% d - % d" % (parent[i], i))


class Graph:
    def __init__(self, V):
        self.V = V
        self.graph = defaultdict(list)

    # Adds an edge to an undirected graph
    def addEdge(self, src, dest, weight):

        # Add an edge from src to dest. A new node is
        # added to the adjacency list of src. The node
        # is added at the beginning. The first element of
        # the node has the destination and the second
        # elements has the weight
        newNode = [dest, weight]
        self.graph[src].insert(0, newNode)

        # Since graph is undirected, add an edge from
        # dest to src also
        newNode = [src, weight]
        self.graph[dest].insert(0, newNode)

    # The main function that prints the Minimum
    # Spanning Tree(MST) using the Prim's Algorithm.
    # It is a O(ELogV) function
    def PrimMST(self):
        # Get the number of vertices in graph
        V = self.V

        # key values used to pick minimum weight edge in cut
        key = []

        # List to store constructed MST
        parent = []

        # minHeap represents set E
        minHeap = Heap()

        # Initialize min heap with all vertices. Key values of all
        # vertices (except the 0th vertex) is is initially infinite
        for v in range(V):
            parent.append(-1)
            key.append(1e7)
            minHeap.array.append(minHeap.newMinHeapNode(v, key[v]))
            minHeap.pos.append(v)

        # Make key value of 0th vertex as 0 so
        # that it is extracted first
        minHeap.pos[0] = 0
        key[0] = 0
        minHeap.decreaseKey(0, key[0])

        # Initially size of min heap is equal to V
        minHeap.size = V

        # In the following loop, min heap contains all nodes
        # not yet added in the MST.
        while minHeap.isEmpty() == False:

            # Extract the vertex with minimum distance value
            newHeapNode = minHeap.extractMin()
            u = newHeapNode[0]

            # Traverse through all adjacent vertices of u
            # (the extracted vertex) and update their
            # distance values
            for pCrawl in self.graph[u]:
                print(pCrawl)

                v = pCrawl[0]

                # If shortest distance to v is not finalized
                # yet, and distance to v through u is less than
                # its previously calculated distance
                if minHeap.isInMinHeap(v) and pCrawl[1] < key[v]:
                    key[v] = pCrawl[1]
                    parent[v] = u

                    # update distance value in min heap also
                    minHeap.decreaseKey(v, key[v])

        printArr(parent, V)


# Driver program to test the above functions
graph = Graph(9)
graph.addEdge(0, 1, 4)
graph.addEdge(0, 7, 8)
graph.addEdge(1, 2, 8)
graph.addEdge(1, 7, 11)
graph.addEdge(2, 3, 7)
graph.addEdge(2, 8, 2)
graph.addEdge(2, 5, 4)
graph.addEdge(3, 4, 9)
graph.addEdge(3, 5, 14)
graph.addEdge(4, 5, 10)
graph.addEdge(5, 6, 2)
graph.addEdge(6, 7, 1)
graph.addEdge(6, 8, 6)
graph.addEdge(7, 8, 7)
graph.PrimMST()

# This code is contributed by Divyanshu Mehta

[7, 8]
[1, 4]
[7, 11]
[2, 8]
[0, 4]
[8, 7]
[6, 1]
[1, 11]
[0, 8]
[8, 6]
[7, 1]
[5, 2]
[6, 2]
[4, 10]
[3, 14]
[2, 4]
[5, 4]
[8, 2]
[3, 7]
[1, 8]
[7, 7]
[6, 6]
[2, 2]
[5, 14]
[4, 9]
[2, 7]
[5, 10]
[3, 9]
 0 -  1
 5 -  2
 2 -  3
 3 -  4
 6 -  5
 7 -  6
 0 -  7
 2 -  8


In [41]:
[7, 8]
[1, 4]
[7, 11]
[2, 8]
[0, 4]
[5, 4]
[8, 2]
[3, 7]
[1, 8]
[5, 14]
[4, 9]
[2, 7]
[5, 10]
[3, 9]
0 = 1
1 = 2
2 = 3
3 = 4

SyntaxError: cannot assign to literal (423456091.py, line 15)